In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")

PyTorch version: 2.5.1
CUDA available: True
CUDA version: 12.4
cuDNN version: 90101


In [2]:
import torch
import torch.nn as nn
import torchvision.models.video as video_models

print("=== System Info ===")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

print("\n=== Test 1: Simple Conv3d ===")
try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Simple 3D conv test
    conv3d = nn.Conv3d(3, 64, kernel_size=3, stride=1, padding=1).to(device)
    
    # Create a small test tensor (batch_size=1, channels=3, depth=8, height=112, width=112)
    test_input = torch.randn(1, 3, 8, 112, 112).to(device)
    
    output = conv3d(test_input)
    print(f"✓ Simple Conv3d test passed! Output shape: {output.shape}")
    
except Exception as e:
    print(f"✗ Simple Conv3d test failed: {e}")

print("\n=== Test 2: R3D18 Model ===")
try:
    # Test the same model you're using
    model = video_models.r3d_18(pretrained=False).to(device)
    
    # Same input size as your video data
    test_input = torch.randn(1, 3, 16, 112, 112).to(device)  # Adjust based on your input size
    
    with torch.no_grad():  # No need for gradients in test
        output = model(test_input)
    
    print(f"✓ R3D18 test passed! Output shape: {output.shape}")
    
except Exception as e:
    print(f"✗ R3D18 test failed: {e}")

print("\n=== Test 3: CPU Fallback ===")
try:
    # Force CPU to see if it's a CUDA-specific issue
    device_cpu = torch.device('cpu')
    model_cpu = video_models.r3d_18(pretrained=False).to(device_cpu)
    test_input_cpu = torch.randn(1, 3, 16, 112, 112).to(device_cpu)
    
    with torch.no_grad():
        output_cpu = model_cpu(test_input_cpu)
    
    print(f"✓ CPU test passed! Output shape: {output_cpu.shape}")
    print("→ This suggests the issue is CUDA/cuDNN specific")
    
except Exception as e:
    print(f"✗ CPU test failed: {e}")
    print("→ This suggests a deeper PyTorch installation issue")

print("\n=== Test 4: Your Exact Model Setup ===")
try:
    # Try to replicate your exact setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Assuming you're using r3d_18 - adjust if different
    model = video_models.r3d_18(pretrained=True)
    
    # Modify for your number of classes if needed
    # model.fc = nn.Linear(model.fc.in_features, YOUR_NUM_CLASSES)
    
    model = model.to(device)
    model.train()  # Set to training mode like in your code
    
    # Test with a batch similar to your training
    batch_size = 2
    test_batch = torch.randn(batch_size, 3, 16, 112, 112).to(device)
    
    output = model(test_batch)
    print(f"✓ Your model setup test passed! Output shape: {output.shape}")
    
except Exception as e:
    print(f"✗ Your model setup test failed: {e}")

print("\n=== Recommendations ===")
if torch.cuda.is_available():
    print("If tests failed:")
    print("1. Try: pip uninstall torch torchvision && pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118")
    print("2. Or try: conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia")
    print("3. Check nvidia-smi and nvcc --version match your PyTorch CUDA version")
else:
    print("CUDA not available - you'll need to run on CPU or fix CUDA installation")

=== System Info ===
PyTorch version: 2.5.1
CUDA available: True
CUDA version: 12.4
cuDNN version: 90101
GPU count: 1
Current GPU: 0
GPU name: NVIDIA GeForce GTX 1080 Ti

=== Test 1: Simple Conv3d ===
Using device: cuda
✓ Simple Conv3d test passed! Output shape: torch.Size([1, 64, 8, 112, 112])

=== Test 2: R3D18 Model ===


/home/luke/miniconda3/envs/wlasl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luke/miniconda3/envs/wlasl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✓ R3D18 test passed! Output shape: torch.Size([1, 400])

=== Test 3: CPU Fallback ===
✓ CPU test passed! Output shape: torch.Size([1, 400])
→ This suggests the issue is CUDA/cuDNN specific

=== Test 4: Your Exact Model Setup ===


/home/luke/miniconda3/envs/wlasl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✓ Your model setup test passed! Output shape: torch.Size([2, 400])

=== Recommendations ===
If tests failed:
1. Try: pip uninstall torch torchvision && pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
2. Or try: conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia
3. Check nvidia-smi and nvcc --version match your PyTorch CUDA version


In [3]:
# Delete specific variables
import gc

# Then clear cache
torch.cuda.empty_cache()
gc.collect()
if torch.cuda.is_available():
    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"GPU memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")

GPU memory allocated: 0.49 GB
GPU memory cached: 0.52 GB


### other tests

In [4]:
import torch
# import torchaudio
# try:
#     import torchcodec
#     print("✓ torchcodec available")
# except ImportError:
#     print("torchcodec not installed")

# print(f"✓ torchaudio version: {torchaudio.__version__}")

In [5]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version in PyTorch: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")

PyTorch version: 2.5.1
CUDA available: True
CUDA version in PyTorch: 12.4
cuDNN version: 90101
